## **Проект "English Score"**

Цель проекта: создать модель классификации фильмов по уровню английского языка.


Задачи проекта:

1. Загрузить данные (расширить дата-сет, если это возможно);
2. Преобразовать и почистить данные;
3. Подготовить данные к моделированию;
4. Моделирование;
5. Оценка качества моделирования.

Автор:

* ФИО - Рожнятовский Г.И.
* email - grinef00@yandex.ru
* telegram - @grinef


Дополнительный комментарий:
* Полностью код выполняется в течение 15 минут.
* на данный момент представлена первая версия кода.

### **Импорты**

In [1]:
# для загрузки данных
import os
import camelot
import ghostscript
import pysrt
import re

# для анализа данных:
import pandas as pd
from pandas import Series
import numpy as np

In [2]:
# предобработка
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Загрузка списка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Grine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Загрузка предварительно обученной модели POS-теггинга
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Grine\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
# для лемматизации
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Grine\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### **Загрузка данных**

#### **Загрузка и подготовка словарей**

In [7]:
# загрузим сначала все слова, и распределим по уровню
directory = r'C:\Users\Grine\Desktop\GitHub\English_level_predict\Oxford_CEFR_level'

# Получение списка всех файлов в директории
file_names = os.listdir(directory)

cefr_vocabulary = {
    'a1': [],
    'a2': [],
    'b1': [],
    'b2': [],
    'c1': []
    } # создадим словарь, в который внесём все уровни слов

# прочитаем все файлы в папке и сгенерируем путь

for file_name in file_names:
    file_path = os.path.join(directory, file_name)      # создадим пути к всем pdf-файлам
    
    tables = camelot.read_pdf(file_path,
                              pages='1-end',
                              flavor='stream',
                              strip_text='\n')          # создадим список всех таблиц
    engish_level = None
    for table in tables:                                # для каждой таблицы
        target_object = table.df.reset_index(drop=True) # обновим индексы
        
        for c_n in target_object.columns:               # и выделим названия столбцов
            for v in target_object[c_n]:                # пройдёмся циклом по каждому значению каждого столбца
                
                value = v.lower()
                if value in cefr_vocabulary.keys():      # если значение столбца есть в ключах словаря
                    engish_level = value                 # то присвоим переменной это значение
                    
                # добавим значение в словарь
                try:
                    cefr_vocabulary[engish_level].append(value.split(' ')[0]) if value != '' else None
                except KeyError: # если ключа нет - продолжим. 
                    continue      

In [8]:
count = 0
for i in cefr_vocabulary.keys():
    amount = len(cefr_vocabulary[i])
    count += amount
    print(f'в категории, {i}, всего {amount} слов')

print(f'Всего слов отобрано: {count}')

в категории, a1, всего 1820 слов
в категории, a2, всего 1694 слов
в категории, b1, всего 1700 слов
в категории, b2, всего 2888 слов
в категории, c1, всего 2657 слов
Всего слов отобрано: 10759


#### **Загрузка данных для  анализа**

In [9]:
subtitle_path = r'C:\Users\Grine\Desktop\GitHub\English_level_predict\english_scores' # путь к файлу, где лежат данные

# Получение списка всех групп-файлов в директории
subtitle_group = os.listdir(subtitle_path) 

# загрузим размеченные фильмы 
movie_labels = pd.read_excel(subtitle_path + '\\' + subtitle_group[0]).iloc[:, 1:]

# преобразуем названия столбцов
movie_labels.columns = [column_name.lower() for column_name in movie_labels.columns]

In [10]:
# переинециализируем путь и снова получим список групп
subtitle_path = subtitle_path + '\\' + subtitle_group[1]
subtitle_group_labels = os.listdir(subtitle_path)[1:] # первый файл '.DS_store' исключим его из дальнейшего парсера

In [11]:
df = pd.DataFrame()
for sub_group in subtitle_group_labels:                                               # для каждой группы папок
    path_files = subtitle_path + '\\' + sub_group                                     # инициализируем новый путь
    files = os.listdir(path_files)                                                    # получим список файлов
    for file_name in files:                                                           # прочтём каждый файл
        df_step = pd.DataFrame({                                                      
          'label':  [sub_group],
          'name': [file_name.split('.s')[0]],
          'subtitle': [pysrt.open(path_files + '\\' + file_name, encoding='latin-1').text]
        }) #  внесём доступные данные о дата-фрейме в нашу таблицу
        
        df = pd.concat([df, df_step]) # соединим результаты
        
df = df.reset_index(drop=True)

In [12]:
# далее, попробуем соединить те фильмы, которые храняться в переменной: "movie_labels" и "df"
# Но перед тем как это сделать, стоит преобразовать данные.
def base_text_transform(data, column) -> Series:
    '''
    На входе функция получает данные (data) и столбец, которые требует преобразования (column)
    На выходе функция возвращает предобработанный столбец, в котором храняться только цифры, буквы и знак пробела -> "_"
    '''
    array = data[column].values # преобразуем в массив
    array_lower = [mean.lower() for mean in array] # приведём массив к нижнему регистру
    array_lower_clean = [re.sub(r'[^a-zA-Z0-9_\s]', '', mean) for mean in array_lower] # оставим только буквы, цифры и пробелы
    array_lower_clean_space = [mean.replace(' ', '_') for mean in array_lower_clean]   # преобразуем пробелы на нижние подчёркивания
    return pd.Series(array_lower_clean_space) # вернём очищенный столбец

In [13]:
df['name_to_merge'] = base_text_transform(df, 'name')                      # преобразуем таблицу с субтитрами
movie_labels['name_to_merge'] = base_text_transform(movie_labels, 'movie') # преобразуем таблицу с метками

In [14]:
df_full = movie_labels.merge(df, how='outer') # создадим дата-фрейм, включающий все записи

### **Подготовка и изучение данных**

#### Изучение данных

In [15]:
df_full.head(4) 

,movie,level,name_to_merge,label,name,subtitle
0,10_Cloverfield_lane(2016),B1,10_cloverfield_lane2016,Subtitles,10_Cloverfield_lane(2016),"<font color=""#ffff80""><b>Fixed & Synced by boz..."
1,10_things_I_hate_about_you(1999),B1,10_things_i_hate_about_you1999,Subtitles,10_things_I_hate_about_you(1999),"Hey!\nI'll be right with you.\nSo, Cameron. He..."
2,A_knights_tale(2001),B2,a_knights_tale2001,Subtitles,A_knights_tale(2001),Resync: Xenzai[NEF]\nRETAIL\nShould we help hi...
3,A_star_is_born(2018),B2,a_star_is_born2018,Subtitles,A_star_is_born(2018),"- <i><font color=""#ffffff""> Synced and correct..."


In [16]:
df_full.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   movie          241 non-null    object
 1   level          241 non-null    object
 2   name_to_merge  292 non-null    object
 3   label          283 non-null    object
 4   name           283 non-null    object
 5   subtitle       283 non-null    object
dtypes: object(6)
memory usage: 13.8+ KB


In [17]:
# изучим пропуски
df_full.isna().sum()

movie            51
level            51
name_to_merge     0
label             9
name              9
subtitle          9
dtype: int64

In [18]:
# у нас есть 2 столбца хранящих названия: name и movie
# надо определить, почему столбцы не смёржились. Чем вызвана такая ошибка?
movie_loss = df_full[df_full['movie'].isna()]
print('\nМассив уникальных названий, которые есть в таблице df: \n',
      movie_loss['name'].values, 
     '\nВсего таких фильмов:', len(movie_loss['name'].values))

name_loss = df_full[df_full['name'].isna()]
print('\nМассив уникальных названий, для которых есть уровень, но нет субтитров: \n',
      name_loss['movie'].values,
     '\nВсего таких фильмов:', len(name_loss['movie'].values))


Массив уникальных названий, которые есть в таблице df: 
 ['SlingShot (2014) WEB.eng' 'Crown, The S01E01 - Wolferton Splash.en.SDH'
 'Crown, The S01E01 - Wolferton Splash.en'
 'Crown, The S01E02 - Hyde Park Corner.en.SDH'
 'Crown, The S01E02 - Hyde Park Corner.en'
 'Crown, The S01E03 - Windsor.en.FORCED'
 'Crown, The S01E03 - Windsor.en.SDH' 'Crown, The S01E03 - Windsor.en'
 'Crown, The S01E04 - Act of God.en.SDH'
 'Crown, The S01E04 - Act of God.en'
 'Crown, The S01E05 - Smoke and Mirrors.en.FORCED'
 'Crown, The S01E05 - Smoke and Mirrors.en.SDH'
 'Crown, The S01E05 - Smoke and Mirrors.en'
 'Crown, The S01E06 - Gelignite.en.SDH' 'Crown, The S01E06 - Gelignite.en'
 'Crown, The S01E07 - Scientia Potentia Est.en.FORCED'
 'Crown, The S01E07 - Scientia Potentia Est.en.SDH'
 'Crown, The S01E07 - Scientia Potentia Est.en'
 'Crown, The S01E08 - Pride & Joy.en.SDH'
 'Crown, The S01E08 - Pride & Joy.en'
 'Crown, The S01E09 - Assassins.en.SDH' 'Crown, The S01E09 - Assassins.en'
 'Crown, The S01E

In [19]:
name_loss

,movie,level,name_to_merge,label,name,subtitle
82,The Secret Life of Pets.en,B2,the_secret_life_of_petsen,NaN,NaN,NaN
106,Up (2009),A2/A2+,up_2009,NaN,NaN,NaN
155,SOMM.Into.the.Bottle.2015.1080p.BluRay.x265-RA...,B2,sommintothebottle20151080pblurayx265rarbgensrt,NaN,NaN,NaN
235,Glass Onion,B2,glass_onion,NaN,NaN,NaN
236,Matilda(2022),C1,matilda2022,NaN,NaN,NaN
237,Bullet train,B1,bullet_train,NaN,NaN,NaN
238,Thor: love and thunder,B2,thor_love_and_thunder,NaN,NaN,NaN
239,Lightyear,B2,lightyear,NaN,NaN,NaN
240,The Grinch,B1,the_grinch,NaN,NaN,NaN


In [20]:
df_full.loc[268] # можно заметить, что объект 155, на самом делел представлен в наших данных, под идексом 266
# поскольку уровень совпадает, индекс 155 можно удалить.
# Все остальные значения уникальные. 

movie                                                          NaN
level                                                          NaN
name_to_merge          sommintothebottle20151080pblurayx265rarbgen
label                                                           B2
name             SOMM.Into.the.Bottle.2015.1080p.BluRay.x265-RA...
subtitle         What-- What is a sommelier?\nA sommelier is a ...
Name: 268, dtype: object

In [21]:
df_full = df_full.drop(index=155, axis=0)

**В результате можно видеть, что у нас есть 8 фильмов, у которых уже присвоена метка класса, но отсуствуют субтитры. Эти субтитры можно догрузить. Но мы удалим этим субтитры. Также удалим пустые субтитры**

In [22]:
index_with_miss_values = df_full[df_full['subtitle'].isin([''])].index.to_list() + df_full[df_full['subtitle'].isna()].index.to_list()
df_full = df_full.drop(index=index_with_miss_values, axis=0).reset_index(drop=True)

In [23]:
df_full['level'].fillna(df_full['label'], inplace=True)

In [24]:
# изучим категоризацию объектов
df_full.reset_index().groupby(['level']).index.agg({'count'})

,count
level,
A2,6
A2/A2+,25
"A2/A2+, B1",5
B1,54
"B1, B2",8
B2,136
C1,23
Subtitles,9


Видно, что есть метки со смешенные классов. Правильно будет сделать так:
    
    1) Для А2/A2+, - можно перенести объекты к метке A2, поскольку их мало. Таким образом мы снизим дисбаланс классов
    2) Для меток A2/A2+, B1, мы можем отнести объект к метке B1.
    3) Для метки B1, B2, можем отнести объект к B1.
 
Такими действиями мы сократим дисбаланс классов.

In [25]:
df_full['level'].replace({'A2/A2+': 'A2',
                          'A2/A2+, B1': 'B1',
                          'B1, B2': 'B1'},
                         inplace=True)

df_full.reset_index().groupby(['level']).index.agg({'count'}) # дисбаланс классов заметно сократился

,count
level,
A2,31
B1,67
B2,136
C1,23
Subtitles,9


#### Подготовка данных

10 фильмов - не имеют разметки, при этом, наблюдается явный дисбаланс классов.
решить эту задачу можно несколькими методами:
    
    1) сгенерировать тексты определнного уровня;
    2) тонко настроить модель, чтобы она учитывала баланс классов;
    
Поскольку время выполнения проекта сильно ограничено, будем использовать второй вариант. 

In [26]:
# там где отсутсвует название name, перенесём его из названия movie
df_full['name'].fillna(df_full['movie'], inplace=True)

In [27]:
# теперь создадим объект, который будем в дальнейшем использовать для анализа 
data = df_full[['name', 'subtitle', 'level']]
data = data.astype('str')

Перед проведением eda нам необходимо очитсить наши тексты. для этого можно использовать несколько технологий:

    1) Стоит очистить тексты от лишних символов, признаков разметки;
    2) затем нужно провести токенизацию текста;
    2) очистить текст от предлогов;
    4) далее стоит провести pos-теггинг и леммматизацию.

In [28]:
class TextCleaning:
    '''
    Класс предназначен для предобработки текста
    '''
    def __init__(self, text):
        self.text = text

    def process_text(self):
        '''
        В субтитрах представленно много "служебных" данных, они представлены в разных видах скобок. Это надо очистить. 
        '''
        # Удаление объектов в квадратных скобках и самих скобок
        text = re.sub(r'\[.*?\]', '', self.text)

        # Удаление объектов в фигурных скобках и самих скобок
        text = re.sub(r'\{.*?\}', '', text)

        # Удаление объектов внутри скобок
        text = re.sub(r'\(.*?\)', '', text)

        # Удаление объектов между <>
        text = re.sub(r'<.*?>', '', text)

        # Удаление лишних пробелов и символов перевода строки
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()

        self.text = text

    def tokenizer(self):
        '''
        Токенизация текста позволит провести все остальные операции с текстом
        '''
        tokens = word_tokenize(self.text)
        self.text = tokens

    def drop_stopwords(self):
        '''
        Удаление стоп-слов позволит повысить качество анализа
        '''
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in self.text if word.lower() not in stop_words]
        self.text = filtered_tokens

    def pos_tagging(self):
        '''
        Пос-теггинг поможет, поскольку в наших словарях есть части речи. это повысит точность коннекта
        '''
        pos_tags = nltk.pos_tag(self.text)
        self.text = pos_tags

    def lemmatization(self):
        '''
        Функция предназначена для лемматизации текста
        '''
        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [
            (lemmatizer.lemmatize(word, pos=self.get_wordnet_pos(tag)), tag)
            for word, tag in self.text
        ]
        self.text = lemmatized_words
    
    @staticmethod
    def get_wordnet_pos(tag):
        '''
        Вспомогательная функция служит для того, чтобы в зависимости от тега вернуть нужную лемму
        '''
        if tag.startswith('J'):
            return nltk.corpus.wordnet.ADJ
        elif tag.startswith('V'):
            return nltk.corpus.wordnet.VERB
        elif tag.startswith('N'):
            return nltk.corpus.wordnet.NOUN
        elif tag.startswith('R'):
            return nltk.corpus.wordnet.ADV
        else:
            return nltk.corpus.wordnet.NOUN

In [29]:
text = df.loc[0]['subtitle']

cleaner = TextCleaning(text)
cleaner.process_text()
cleaner.tokenizer()
cleaner.drop_stopwords()
cleaner.pos_tagging()
cleaner.lemmatization()

print(cleaner.text[:25])

[('-', ':'), ('-', ':'), ('Little', 'JJ'), ('girl', 'NN'), ('?', '.'), ("'m", 'VBP'), ('policeman', 'JJ'), ('.', '.'), ('Little', 'JJ'), ('girl', 'NN'), ('.', '.'), ("n't", 'RB'), ('afraid', 'JJ'), (',', ','), ('okay', 'VB'), ('?', '.'), ('Little', 'JJ'), ('girl', 'NN'), ('.', '.'), ('Oh', 'NNP'), ('God', 'NNP'), ('.', '.'), ('-', ':'), ('-', ':'), ('Man', 'NN')]


In [30]:
%%time
# теперь применим наш класс к нашему дата-фрейму:
def clean(x):
    cleaner = TextCleaning(x)
    cleaner.process_text()
    cleaner.tokenizer()
    cleaner.drop_stopwords()
    cleaner.pos_tagging()
    cleaner.lemmatization()
    return cleaner.text

data['sub_clean'] = data.subtitle.apply(lambda x: clean(x))

Wall time: 40.7 s


На выходе, мы получаем список, состоящий из кортежей, где в первом элементе кортежа представлена лема, а во втором элементе кортежа представлен тегг.

Далее предлагается написать ещё одну функцию, которая будет распаршивать резюмирующий объект, и возвращать несколько списков:
    
    1) Количество слов;
    2) Количество слов относящихся к A1, A2, B1, B2, C1, C2, кол-во слов не представленных в словаре;
    3) Количество теггов, относящихся к A1, A2, B1, B2, C1, C2, кол-во слов не представленных в словаре;
    4) Количество знаков.
   


In [31]:
# создадим словарь с приблизительными пос-теггами по уровню английского
pos_tegg_dict = {
    'a1': ['NN', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ'],
    'a2': ['DT', 'PRP', 'RB'],
    'b1': ['CC', 'IN', 'MD', 'WRB'],
    'b2': ['RP', 'JJR'],
    'c1': ['JJS'],
    'c2': ['FW']
}

In [32]:
def count_tags(input_dataframe: pd.DataFrame,
               column: str,
               tag_dict: dict,
               word_dict: dict) -> pd.DataFrame:
    tag_columns = ['a1_teg', 'a2_teg', 'b1_teg', 'b2_teg', 'c1_teg', 'c2_teg']
    word_columns = ['a1_word', 'a2_word', 'b1_word', 'b2_word', 'c1_word', 'c2_word']

    input_dataframe = input_dataframe.reset_index(drop=True)

    inversed_tag_dict = {value: key for key, lst in tag_dict.items() for value in lst}
    inversed_word_dict = {value: key for key, lst in word_dict.items() for value in lst}

    row_dict = {}

    for i, list_pair in enumerate(input_dataframe[column]):
        tag_counters = {}.fromkeys(tag_dict.keys(), 0)
        word_counters = {}.fromkeys(inversed_word_dict.keys(), 0)
        word_counters |= {'words_without_tag': 0, 'non-words': 0}
        for word, tag in list_pair:
            word_ = word.lower()
            if inversed_tag_dict.get(tag):
                tag_counters[inversed_tag_dict[tag]] += 1
            if inversed_word_dict.get(word_):
                word_counters[inversed_word_dict[word_]] += 1
            else:
                # print(word_)
                # non words:
                non_word = re.search(r'\W+', word_)
                if non_word is not None:
                    word_counters['non-words'] += 1
                    # print(non_word.group())
                else:
                    i_think_it_is_word = re.search(r'\w+', word_)
                    if i_think_it_is_word:
                        word_counters['words_without_tag'] += 1
        else:
            row_dict |= {i: [
                {f'{k}_tag': v for k, v in tag_counters.items()},
                {f'{k}_word': v for k, v in word_counters.items() if v != 0}
            ]}
    new_columns = []
    for val in row_dict.values():
        new_dict = {}
        for d in val:
            new_dict |= {**d}
        new_columns.append(new_dict)
    return pd.concat([input_dataframe, pd.DataFrame(new_columns)], axis=1)

In [33]:
data = count_tags(data, 'sub_clean', pos_tegg_dict, cefr_vocabulary)

In [34]:
# распарсим данные
data['sub'] = [' '.join([v[0].lower() for v in value if v[0] not in '/.,<>!*&^%($#)@_!:?....-.-']) for value in data['sub_clean']]

In [35]:
data.drop(columns=['name', 'subtitle', 'sub_clean'], axis=1, inplace=True)

### **Подготовка данных и моделирование**

In [36]:
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import randint as sp_randint
from sklearn.decomposition import PCA


In [37]:
def train_test_valid_split(data):
    '''
    Функция предназначена для разделения данных на test, train и valid;
    Функция нужна для реперезентативности выборки обучения. Учитывает дисбаланс классов;
    '''
    np.random.seed(42)
    
    data = data.reset_index()
    
    test = data[data['level'] == 'Subtitles'].reset_index(drop=True)
    data = data[data['level'] != 'Subtitles']
    
    stats = data.groupby('level')['level'].agg({'count'}).reset_index()
    stats['target_count_indexes'] = (stats['count'] * 0.8).astype('int')
    
    d = {} # создадим словари со всеми индексами
    for level in data['level'].unique():
        d.update({level: []})
    
    for level, idx in zip(data['level'], data['index']):
        d[level].append(idx)
    
    d_count = {} # словарь для отбора кол-ва индексов, которые нужно отобрать для каждой группы
    
    for level, count in zip(stats['level'], stats['target_count_indexes']):
        d_count.update({level: count})
    
    idx_lists = list() # создадим список с индексами
    for key, n in d_count.items(): # распарсим словарь с количеством
        lists_means = random.sample(d[key], n) # сделаем рандомный выбор значений, для каждого ключа, для n - кол-ва
        idx_lists.append(lists_means) # добавим в конец списка индексы
        
    index_target = list()
    for values in idx_lists:
        for v in values:
            index_target.append(v)
    
    train = data[data['index'].isin(index_target)].reset_index(drop=True)
    valid = data[~data['index'].isin(index_target)].reset_index(drop=True)
    
    train.rename(columns={'level': 'label_class'}, inplace=True)
    test.rename(columns={'level': 'label_class'}, inplace=True)
    valid.rename(columns={'level': 'label_class'}, inplace=True)
    
    return train.drop(columns='index', axis=1), test.drop(columns='index', axis=1), valid.drop(columns='index', axis=1)

train, test, valid = train_test_valid_split(data)

In [38]:
print('Тренировочная выборка:', round((len(train)/len(data))*100), '%')
print('Валидационная выборка:', round((len(valid)/len(data))*100), '%')
print('Тестовая выборка:', round((len(test)/len(data))*100), '%')

Тренировочная выборка: 76 %
Валидационная выборка: 20 %
Тестовая выборка: 3 %


In [39]:
# подготовим колонки
text_column = 'sub'
target = 'label_class'
numeric_columns = [column for column in train.columns if column not in text_column and column not in target]

mms = MinMaxScaler()
le = LabelEncoder()
tf_idf = TfidfVectorizer()

train[numeric_columns] = mms.fit_transform(train[numeric_columns])
valid[numeric_columns] = mms.transform(valid[numeric_columns])
test[numeric_columns] = mms.transform(test[numeric_columns])

train[target] = le.fit_transform(train[target])
valid[target] = le.transform(valid[target])

tfidf_matrix = tf_idf.fit_transform(train[text_column])
tfidf_df_train = pd.DataFrame(tfidf_matrix.toarray(), columns=tf_idf.get_feature_names())

tfidf_matrix = tf_idf.transform(valid[text_column])
tfidf_df_valid = pd.DataFrame(tfidf_matrix.toarray(), columns=tf_idf.get_feature_names())

tfidf_matrix = tf_idf.transform(test[text_column])
tfidf_df_test = pd.DataFrame(tfidf_matrix.toarray(), columns=tf_idf.get_feature_names())

train = train.drop(columns='sub', axis=1)
train = pd.concat([train, tfidf_df_train], axis=1)

valid = valid.drop(columns='sub', axis=1)
valid = pd.concat([valid, tfidf_df_valid], axis=1)

test = test.drop(columns='sub', axis=1)
test = pd.concat([test, tfidf_df_test], axis=1)

target_train = train[target]
target_valid = valid[target]

features_train = train.drop(columns=target, axis=1).fillna(0)
features_valid = valid.drop(columns=target, axis=1).fillna(0)
features_test = test.drop(columns=target, axis=1).fillna(0)


# Создание pipeline без параметров модели
pipeline = Pipeline([
    ('model', CatBoostClassifier())
])

# Задание сетки параметров для случайного поиска
parameters = {
    'model__iterations': [500, 1000, 1500],
    'model__loss_function': ['MultiClass', 'Logloss'],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__depth': [4, 6, 8],
    'model__l2_leaf_reg': [1, 3, 5],
    'model__border_count': [32, 64, 128],
    'model__verbose': [100]
}

# Поиск наилучших параметров с использованием RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, parameters, cv=3, n_iter=10, random_state=42)
random_search.fit(features_train, target_train)

# Получение наилучшей модели
best_model = random_search.best_estimator_


# Предсказание с использованием лучшей модели
y_pred = best_model.predict(features_valid)

# Оценка точности предсказаний
accuracy = accuracy_score(target_valid, y_pred)

0:	learn: 1.3813014	total: 266ms	remaining: 6m 38s
100:	learn: 1.0145394	total: 11.3s	remaining: 2m 35s
200:	learn: 0.7956311	total: 22.2s	remaining: 2m 23s
300:	learn: 0.6532248	total: 33.2s	remaining: 2m 12s
400:	learn: 0.5431636	total: 44.2s	remaining: 2m 1s
500:	learn: 0.4546312	total: 55.2s	remaining: 1m 50s
600:	learn: 0.3737064	total: 1m 6s	remaining: 1m 38s
700:	learn: 0.3006440	total: 1m 16s	remaining: 1m 27s
800:	learn: 0.2413312	total: 1m 27s	remaining: 1m 16s
900:	learn: 0.1971673	total: 1m 38s	remaining: 1m 5s
1000:	learn: 0.1652286	total: 1m 49s	remaining: 54.7s
1100:	learn: 0.1399913	total: 2m	remaining: 43.7s
1200:	learn: 0.1205215	total: 2m 11s	remaining: 32.8s
1300:	learn: 0.1057819	total: 2m 22s	remaining: 21.8s
1400:	learn: 0.0939148	total: 2m 33s	remaining: 10.8s
1499:	learn: 0.0841683	total: 2m 44s	remaining: 0us
0:	learn: 1.3822102	total: 141ms	remaining: 3m 31s
100:	learn: 1.0594788	total: 12.7s	remaining: 2m 55s
200:	learn: 0.8798502	total: 25.1s	remaining: 2m 

In [40]:
print(accuracy)

0.6666666666666666


Резюмирующая точность оказалась 66% - это может говорить о трёх проблемах:
    
    1) Плохо выполнен процесс предобработки, подготовки данных;
    2) Данные описаны неточно (стоит провести дополнительную обработку размерности входящего массива, проработать веса);
    3) Данных недостаточно для обучения (стоит расширить дата-сет).
    
    
После ревью планируется:
    
    1) Исправить ошибки, если они есть;
    2) попробовать другие подходы к подготовке (например, сделать оценку частнотности, по уникальным словам, а не по всем или провести лемматизацию слов, представленных в словаре, для повышения точности соединения)
    3) расширить данные;

**Параметры дающие accuracy 66%**

    {
    'memory': None,
     'steps': [('model', <catboost.core.CatBoostClassifier at 0x21616165fd0>)],
     'verbose': False,
     'model': <catboost.core.CatBoostClassifier at 0x21616165fd0>,
     'model__iterations': 500,
     'model__learning_rate': 0.1,
     'model__depth': 8,
     'model__l2_leaf_reg': 5,
     'model__loss_function': 'MultiClass',
     'model__border_count': 128,
     'model__verbose': 100
     }